# 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [ ]:
!wget -q http://apache.mirrors.pair.com/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz



In [ ]:
!ls

drive  sample_data  spark-2.4.5-bin-hadoop2.7.tgz


In [ ]:
!tar -xvf spark-2.4.5-bin-hadoop2.7.tgz


spark-2.4.5-bin-hadoop2.7/
spark-2.4.5-bin-hadoop2.7/licenses/
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-jtransforms.html
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-zstd.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-zstd-jni.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-xmlenc.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-vis.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-spire.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-sorttable.js.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-slf4j.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-scopt.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-scala.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-sbt-launch-lib.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-respond.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-reflectasm.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-pyrolite.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-py4j.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-protobuf.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-pmml-model

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ratings_int_new.csv to ratings_int_new.csv


In [ ]:
import re;
import os;
import sys;
from pyspark import SparkConf, SparkContext, HiveContext;
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating;

In [ ]:
from heapq import nlargest;
sc = SparkContext(master="local",appName="Spark Demo");
sqlContext = HiveContext(sc);

In [ ]:
ratings = sc.textFile("ratings_int_new.csv");

In [ ]:
data_header = ratings.take(1); 

In [ ]:
print(data_header)

['6,1,1']


In [ ]:
for x in ratings.collect():
  print (x)

KeyboardInterrupt: ignored

In [ ]:
#ratings = ratings.filter(lambda line: line != data_header).map(laAmbda line: line.split(",")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),float(tokens[2])))
ratings = ratings.map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),float(tokens[2])))

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving business_int_new.csv to business_int_new.csv


In [ ]:
business = sc.textFile("business_int_new.csv");

In [ ]:
data_header = business.take(1)[0];

In [ ]:
print(data_header)

21744,808


In [ ]:
business = business.map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),tokens[1])).cache()

In [ ]:
new_user_ID = 3
new_user_ratings = [
     (3,10,4),
     (3,14,3),
     (3,19,3),
     (3,22,4),
     (3,32,4),
     (3,44,1),
     (3,53,1),
     (3,63,3),
     (3,70,5),
     (3,104,4)
    ]

In [ ]:
new_user_ratings_RDD = sc.parallelize(new_user_ratings)

In [ ]:
ratings_new_user = ratings.union(new_user_ratings_RDD);

In [ ]:
train, test = ratings_new_user.randomSplit([0.8, 0.2])

In [ ]:
rank=10;
numIterations=10;


In [ ]:
type(ratings_new_user)

pyspark.rdd.RDD

In [ ]:
model = ALS.train(ratings_new_user, rank,numIterations);

In [ ]:
new_user_ratings_ids = map(lambda x: x[1], new_user_ratings)
new_user_unrated_RDD = (business.filter(lambda x: x[0] not in new_user_ratings_ids).map(lambda x: (new_user_ID, x[0])))

In [ ]:
new_user_recommendations_RDD = model.predictAll(new_user_unrated_RDD)
new_user_recommendations_RDD=new_user_recommendations_RDD.map(lambda x:(x.product,x.rating));
result=new_user_recommendations_RDD.join(business);
result=result.map(lambda a:(a[0],a[1][0],a[1][1]));
result.saveAsTextFile("joinedresult");

#sc.stop();


In [ ]:
for x in result.collect():
  print (x)

Streaming output truncated to the last 5000 lines.
(28121, 3.60563559575588, 'Flores Air Conditioning and Heating')
(20551, 2.888427961161371, "Lowe's Home Improvement")
(4503, 0.9198572726311831, "Denny's")
(24191, -0.23942671716620262, 'Eighty-Eight Broadway')
(23621, 0.6966195501767473, "Shane's Rib Shack")
(16881, 3.344037594342282, 'Evos')
(15127, 0.3654056211372594, 'Cheep Cheep Movers')
(19599, 1.2598912403456484, 'Sonic Drive-In')
(20419, 2.1748489033957483, 'Lemontree')
(1067, 1.9326686693180208, 'Folger Kia East Charlotte')
(29751, -0.33425940584325525, 'Flight')
(31933, 0.5304316374351874, 'Banana Republic Factory Store')
(23179, -0.3518351769857203, 'Cadillac Ranch')
(14933, 0.2010534357260646, "Rosemary's Restaurant")
(9693, 1.1587870996593095, 'Toppers Pizza')
(9015, 1.1101349730908705, "Houlihan's")
(24301, 0.36989673506043863, 'Scores Rotisserie & Ribs')
(23695, 0.1613747588887675, 'Boulevard Cafe')
(9759, 1.1712759684638785, 'Goodwill Industries of the Southern Piedmon